In [1]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import requests
import folium

In [2]:
distance_df = pd.read_csv('../data/osrm_distance_matrix.csv', index_col=0)

In [3]:
distance_mat = distance_matrix = distance_df.values

In [4]:
locations = list(distance_df.index)

In [5]:
# Display the matrix
print("Distance Matrix (km):")
print(distance_matrix)

print("\nLocations:")
print(locations)

Distance Matrix (km):
[[  0.      53.5289 117.1457  76.1327 102.9948 132.6988 103.9614  70.8661
   62.8791  39.612  131.9956 111.311  116.6007  98.5076 113.1894  71.1478
  131.9508  31.3265 157.3869  45.4115  73.6527  87.7496 116.8822 123.2267
  103.262   73.9666  72.0033 103.8198]
 [ 53.5289   0.     109.7074  79.4945 157.0729 169.4899 140.767   95.8095
   96.4586  73.1916 163.2395 144.8905 123.1707 132.0871 144.4334  96.0912
  156.1096  25.0779 149.9486  37.9732  87.935  125.6694 150.4617 154.4707
  136.8415 107.5461  96.9467 135.9846]
 [117.1457 109.7074   0.      57.749  187.2679 169.9369 119.0216 102.7374
  113.4241 106.7332 141.4941 132.9629 101.4253 140.3003 122.688  103.0191
  134.3642  92.3079  60.8041  71.743   72.3273 103.924  145.5504 132.7253
  142.6954 145.6764 103.8745 114.2391]
 [ 76.1327  79.4945  57.749    0.     129.6105 112.2795  61.3642  45.08
   61.1097  68.9922  83.8367  75.3055  43.7679  82.6429  65.0306  45.3616
   76.7068  62.0714  98.0133  41.5064  14.6699  4

In [6]:
# Create Routing Index Manager
manager = pywrapcp.RoutingIndexManager(
    len(distance_matrix), 1, 0
)  # Single vehicle, depot at index 0

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)


# Distance callback function
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(distance_matrix[from_node][to_node])


# Register the callback
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Search Parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# Solve
solution = routing.SolveWithParameters(search_parameters)

# Print solution with total distance
if solution:
    print("\nOptimal Route:")
    index = routing.Start(0)
    total_distance = 0
    route_order = []

    while not routing.IsEnd(index):
        route_order.append(locations[manager.IndexToNode(index)])
        next_index = solution.Value(routing.NextVar(index))
        total_distance += distance_matrix[manager.IndexToNode(index)][
            manager.IndexToNode(next_index)
        ]
        index = next_index

    route_order.append(locations[manager.IndexToNode(index)])  # Return to depot

    print(" -> ".join(route_order))
    print(f"Total Distance: {total_distance:.2f} km")
else:
    print("No solution found.")


Optimal Route:
Nochchiyagama -> Anuradapura -> Medawachchiya -> Thirappane -> Aralaganwila -> Medirigiriya -> Dambulla -> Dewahuwa -> Polpithigama -> Polpitigama -> Rambawa -> Kumbukgate -> Ibbagamuwa -> Wellawa -> Kurunagala -> Pothuhera -> Ikiriyagolla -> Polgahawela -> Alawwa -> Narammala -> Kithalawa -> Chilaw -> Walipennagahamula -> Siyambalagaskotuwa -> Katupotha -> Minuwangate -> Maho -> Galgamuwa -> Nochchiyagama
Total Distance: 743.70 km


In [7]:
gps_df = pd.read_csv('../data/data_gps.csv')

In [8]:
unique_locations = gps_df.drop_duplicates(subset=["ADDRESS"])[
    ["ADDRESS", "LATITUDE", "LONGITUDE"]
]

In [9]:
unique_locations.iloc[1]["ADDRESS"]

'Medawachchiya'

In [10]:
towns = {
    unique_locations.iloc[i]["ADDRESS"]: [
        unique_locations.iloc[i]["LATITUDE"],
        unique_locations.iloc[i]["LONGITUDE"],
    ]
    for i in range(len(unique_locations))
}

In [11]:
towns

{'Nochchiyagama': [8.2640142, 80.2105099],
 'Medawachchiya': [8.5387775, 80.492996],
 'Medirigiriya': [8.1423253, 80.971342],
 'Dambulla': [7.8742031, 80.6510917],
 'Chilaw': [7.5765074, 79.7956755],
 'Walipennagahamula': [7.4307776, 79.9298775],
 'Wellawa': [7.5581214, 80.3664063],
 'Polpithigama': [7.8172167, 80.4049605],
 'Maho': [7.8239143, 80.273567],
 'Galgamuwa': [7.9956682, 80.2706117],
 'Alawwa': [7.2933377, 80.2387243],
 'Narammala': [7.43073, 80.2143788],
 'Ibbagamuwa': [7.5479128, 80.4496417],
 'Katupotha': [7.53362, 80.1875737],
 'Pothuhera': [7.4198002, 80.3282461],
 'Polpitigama': [7.8204405, 80.4009071],
 'Ikiriyagolla': [7.3374134, 80.4016614],
 'Anuradapura': [8.3681682, 80.43998206],
 'Aralaganwila': [7.782607, 81.184693],
 'Thirappane': [8.2157151, 80.5229819],
 'Dewahuwa': [7.8456614, 80.5737567],
 'Kumbukgate': [7.6760968, 80.424747],
 'Kithalawa': [7.4607943, 80.109762],
 'Polgahawela': [7.3352583, 80.3002463],
 'Siyambalagaskotuwa': [7.4980356, 80.1816287],
 'Mi

In [12]:
# Function to fetch actual road path using OSRM
def get_actual_road_path(town_coords):
    """
    Fetch the actual road path from OSRM API.
    :param town_coords: List of coordinates for the towns in the route.
    :return: GeoJSON road path coordinates.
    """
    # Format coordinates for OSRM API
    coords = ";".join([f"{lon},{lat}" for lat, lon in town_coords])

    # OSRM API URL
    url = f"http://router.project-osrm.org/route/v1/car/{coords}?overview=full&geometries=geojson"

    # Make the API request
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "routes" in data and len(data["routes"]) > 0:
            coordinates = data["routes"][0]["geometry"]["coordinates"]
            print(coordinates)
            return coordinates
    return None


# Visualize routes
def visualize_routes(route, towns):
    """
    Visualize the actual road path on a map.
    :param route: List of town names in the route.
    :param towns: Dictionary of town names and their coordinates.
    :return: Folium map object.
    """
    # Create a base map centered around Kiribathkumbura
    kiribathkumbura_coords = towns["Nochchiyagama"]
    folium_map = folium.Map(location=kiribathkumbura_coords, zoom_start=10)

    # Get coordinates for the towns in the route
    route_coords = [towns[town] for town in route]
    

    # Fetch the actual road path
    road_path = get_actual_road_path(route_coords)
    print(road_path)

    # Plot the road path on the map
    if road_path:
        folium.PolyLine(
            [(lat, lon) for lon, lat in road_path], color="blue", weight=5, opacity=0.7
        ).add_to(folium_map)
    else:
        print("Could not fetch road path from OSRM.")

    # Add markers for each town
    for town in route:
        coord = towns[town]
        folium.Marker(location=coord, popup=town).add_to(folium_map)

    return folium_map

In [14]:
route_order

['Nochchiyagama',
 'Anuradapura',
 'Medawachchiya',
 'Thirappane',
 'Aralaganwila',
 'Medirigiriya',
 'Dambulla',
 'Dewahuwa',
 'Polpithigama',
 'Polpitigama',
 'Rambawa',
 'Kumbukgate',
 'Ibbagamuwa',
 'Wellawa',
 'Kurunagala',
 'Pothuhera',
 'Ikiriyagolla',
 'Polgahawela',
 'Alawwa',
 'Narammala',
 'Kithalawa',
 'Chilaw',
 'Walipennagahamula',
 'Siyambalagaskotuwa',
 'Katupotha',
 'Minuwangate',
 'Maho',
 'Galgamuwa',
 'Nochchiyagama']

In [15]:
# Visualize the route
map_object = visualize_routes(route_order, towns)

[[80.210328, 8.263934], [80.210446, 8.26367], [80.210835, 8.26385], [80.212091, 8.264438], [80.215307, 8.265912], [80.216566, 8.266473], [80.221407, 8.268629], [80.221558, 8.268697], [80.223288, 8.269467], [80.223468, 8.269547], [80.224286, 8.269912], [80.226199, 8.270763], [80.227337, 8.27127], [80.227729, 8.271445], [80.229255, 8.272022], [80.229379, 8.272069], [80.230915, 8.27265], [80.231415, 8.272839], [80.231651, 8.272929], [80.233865, 8.273768], [80.235082, 8.27423], [80.235266, 8.274296], [80.238244, 8.275423], [80.240149, 8.276143], [80.242369, 8.277001], [80.245102, 8.278057], [80.246139, 8.278457], [80.248149, 8.279233], [80.248899, 8.2795], [80.250437, 8.280046], [80.250741, 8.280116], [80.252186, 8.280394], [80.252697, 8.280489], [80.255284, 8.281525], [80.256851, 8.282153], [80.258338, 8.282718], [80.258462, 8.282765], [80.260158, 8.282791], [80.262399, 8.283674], [80.264157, 8.28434], [80.264501, 8.284393], [80.264706, 8.284393], [80.265514, 8.284284], [80.265983, 8.2843

In [16]:
map_object